In [ ]:
!pip install datasets evaluate transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━

# Preprocessing

In [ ]:
from datasets import Dataset, load_dataset

In [ ]:
raw_datasets_crypto = load_dataset("SocialGrep/reddit-crypto-aug-2021", 'comments')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset reddit-crypto-aug-2021 downloaded and prepared to /root/.cache/huggingface/datasets/SocialGrep___reddit-crypto-aug-2021/comments/1.0.0/a1ff130b46f2ea608c366e39d219bd90ff9f856ca60219fac5e0314fed9ff1d9. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Filter out cryptocurrency subreddit comments

raw_datasets_cryptocurrency= raw_datasets_crypto['train'].filter(lambda x: x['subreddit.name'] == 'cryptocurrency')

# Clean the dataset
# Won't care about removing newline symbols for example, as we want them to be included in the generated text as well (to give the generated comments more realistic appearance)
# The filter is not perfect, but should get rid of 99% unnecessary posts.
dataset_cryptocurrency = raw_datasets_cryptocurrency.filter(lambda x: x['body'] not in ['[deleted]', '[removed]'] \
                                                                 and 'I am a bot' not in x['body'] \
                                                                 and '![gif]' not in x['body'] \
                                                                 and 'http://' not in x['body'] \
                                                                 and 'https://' not in x['body'] \
                                                                 and '![img]' not in x['body'])

Filter:   0%|          | 0/3756097 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3301330 [00:00<?, ? examples/s]

In [ ]:
# Select 50 000 random comments

crypto_50000 = dataset_cryptocurrency.shuffle(seed=42).select([i for i in range(50_000, 100_000)])

In [ ]:
# Save just in case

crypto_50000.save_to_disk('crypto_data')
crypto_50000

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw', 'created_utc', 'permalink', 'body', 'sentiment', 'score'],
    num_rows: 50000
})

# Fine-tuning

In [ ]:
from transformers import AutoTokenizer, GPT2Tokenizer

checkpoint = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def tokenize_function(examples):
    comments = []
    for comment in examples["body"]:
      comments.append(tokenizer.eos_token + comment + tokenizer.eos_token)

    return tokenizer(comments, truncation=True)

crypto_tokenized = crypto_50000.map(tokenize_function, batched=True, remove_columns=["type", "id", "subreddit.id", "subreddit.name", "subreddit.nsfw", "created_utc", "permalink", "body", "sentiment", "score"])
crypto_tokenized

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 50000
})

In [ ]:
from transformers import DataCollatorForLanguageModeling, GPT2LMHeadModel, TrainingArguments, Trainer

model = GPT2LMHeadModel.from_pretrained(checkpoint)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments("training", per_device_train_batch_size=4, max_steps=12500, fp16=True) # fp16 makes the training more memory efficient

trainer = Trainer(
    model,
    training_args,
    train_dataset=crypto_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,4.665700
1000,4.404900
1500,4.421200
2000,4.394100
2500,4.380000
3000,4.331200
3500,4.295200
4000,4.293400
4500,4.297100
5000,4.293300


TrainOutput(global_step=12500, training_loss=4.2670934765625, metrics={'train_runtime': 1532.1362, 'train_samples_per_second': 32.634, 'train_steps_per_second': 8.159, 'total_flos': 1147759953408000.0, 'train_loss': 4.2670934765625, 'epoch': 1.0})

In [ ]:
# Saving model and tokenizer for inference

trainer.save_model('crypto')
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.json',
 'tokenizer/merges.txt',
 'tokenizer/added_tokens.json')

# Inference

In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model = 'crypto', tokenizer=tokenizer)
sentences = generator("Bitcoin is going to the", max_length = 150, num_return_sequences=10)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
sentences

[{'generated_text': "Bitcoin is going to the moon? I never thought about it, but a bit concerned how much ETH in the long term will keep it there. I haven't even looked past the hype of the project. And I'm not yet convinced that Bitcoin has the same potential as Ethereum, let alone the future. So there you go! \n\nHope there will be more info. Will just keep an eye out, especially for the comments you see above. I would also like to make some predictions if there ever is a crash. If it happens, it is going to show a real bubble when it actually comes out. So some investors will not do it right. \n\nBut it's just a matter of time for them to start coming up"},
 {'generated_text': 'Bitcoin is going to the moon someday 💚🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🔚🆞\U0001f2de🚀🚀🚀🚀🚀🚀🚀🛀🛀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🎆🚀🎆�'},
 {'generated_text': 'Bitcoin is going to the moon in time. Let’s go! 🙏�🏼🏼\u200d♂️\n\nGood luck and keep flying 🙏�🏼🍻🍻� boasts.\n\nI’m sure that’s the only way for us to go to the moon in time 😂😂👍♂️🏼🍻 🍻🍻🍻🍻🍻🍻🍻\u200d